In [121]:
import numpy as np
import argparse
import math
import random
import matplotlib.pyplot as plt
import time
import sys
import os
from arm import arm
from UCB import UCB
from posterior import posterior
import math
import random
import pandas as pd

## add the relative directory to path to make module load easier
#os.path.abspath('')
curPath = globals()['_dh'][0]
#curPath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(os.path.join(curPath, "Candidate4/", "code/"))
sys.path.append(os.path.join(curPath, "Candidate1/"))

## load self defined modules
#from Candidate1 import test_expert1
import test_expert1
import test_random_forest_svc

In [122]:
import pandas as pd
import numpy as np

print("Preprocessing dataset for Expert 2 (random forest) ... \n")
        # Please modify the path depending on where you put this script
X_test = pd.read_csv('./Candidate4/data/test_rf.csv')
        # drop the useless features
        # 不能直接fillna 必须一个一个病人单独fillna 因为每个病人的数据是独立的
        # X_test.fillna(method = 'bfill', inplace = True)
        # X_test.fillna(method = 'ffill', inplace = True)
        
print("Filling in the NaN values ... ")
num_patients = len(X_test['Patient_id'].unique())
for i in np.arange(num_patients):
 
    X_test.loc[ X_test['Patient_id'] ==  i] =  X_test.loc[ X_test['Patient_id'] == i].fillna(method = 'bfill')
    X_test.loc[ X_test['Patient_id'] == i] =  X_test.loc[ X_test['Patient_id'] == i].fillna(method = 'ffill')
#if not X_test.isnull().values.any().any():
    #print("Have filled all the NaN values successfully! ")     

Preprocessing dataset for Expert 2 (random forest) ... 

Filling in the NaN values ... 


In [118]:
    #X = X_test.drop(['SepsisLabel'],  axis =1 )
X = X_test.drop(['EtCO2', 'BaseExcess','HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos',
    'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct' ,'Lactate','Magnesium','Phosphate',
    'Potassium', 'Bilirubin_total', 'TroponinI','Hct', 'Hgb','PTT',  'WBC', 'Fibrinogen', 'Platelets',
    "Glucose", 'Unit1', "Unit2", "HospAdmTime", 'DBP', 'SBP' ,'time' ], axis = 1 )  

#     X.isnull().values.any().any()

In [119]:
X.to_csv ('test_set_filled.csv', index = False, header=True)

In [120]:
X.loc[test['Patient_id'] == 46].head()

,HR,O2Sat,Temp,MAP,Resp,Age,Gender,ICULOS,SepsisLabel,Patient_id
1767,62.5,100.0,35.17,56.0,NaN,40.13,1,1,0,46
1768,62.5,100.0,35.17,56.0,NaN,40.13,1,2,0,46
1769,65.0,100.0,35.17,53.0,NaN,40.13,1,3,0,46
1770,72.0,99.5,35.17,50.5,NaN,40.13,1,4,0,46
1771,73.0,100.0,35.39,54.0,NaN,40.13,1,5,0,46


In [123]:
X_testpd = pd.read_csv('test_set_filled.csv')
    
    

In [109]:
X_testpd.loc[X_testpd['Patient_id'] ==46] 

,HR,O2Sat,Temp,MAP,Resp,Age,Gender,ICULOS,SepsisLabel,Patient_id
1767,62.5,100.0,35.17,56.0,NaN,40.13,1,1,0,46
1768,62.5,100.0,35.17,56.0,NaN,40.13,1,2,0,46
1769,65.0,100.0,35.17,53.0,NaN,40.13,1,3,0,46
1770,72.0,99.5,35.17,50.5,NaN,40.13,1,4,0,46
1771,73.0,100.0,35.39,54.0,NaN,40.13,1,5,0,46
...,...,...,...,...,...,...,...,...,...,...
1909,103.0,93.0,37.78,63.0,NaN,40.13,1,143,1,46
1910,100.0,93.0,37.78,62.0,NaN,40.13,1,144,1,46
1911,100.0,95.0,37.78,58.0,NaN,40.13,1,145,1,46
1912,104.0,95.0,37.78,65.0,NaN,40.13,1,146,1,46


In [113]:
    training = pd.read_csv('./Candidate4/data/train_rf.csv')
    testing = pd.read_csv('./Candidate4/data/test_rf.csv')
#     print(isinstance(train, pd.DataFrame))
#     print(isinstance(test, pd.DataFrame))

    # 34285 training samples
    T= len(training['Patient_id'].unique())
    ## 6051 testing samples
    T1 = len(testing['Patient_id'].unique())
    print(T,T1)

34285 6051


In [130]:
pd.options.mode.chained_assignment = None  # default='warn'

In [131]:
    for i in np.arange(T1):
        
        #X_filled = X_train.loc[X_train['Patient_id'] ==  i]
        X = X_testpd.loc[X_testpd['Patient_id'] ==i]        
        if X.isnull().values.any():
            
            for Nan_col in X.columns[X.isna().any()].tolist():
                age =  X.iloc[0]['Age']
                gender = X.iloc[0]['Gender']
                # fill in the mean with the mean from patients with similar age and same gender
                X_testpd.loc[abs(X_testpd['Age'] -age)<= 1].loc[X_testpd['Gender'] ==   gender][Nan_col].mean()
                X[Nan_col].fillna(X_testpd[Nan_col].mean(),inplace = True)
                
            X_testpd.loc[X_testpd['Patient_id'] ==  i] = X 
            X_filled = X_testpd.loc[X_testpd['Patient_id'] ==i]   
            if not X_filled.isnull().values.any():
                #if i%10 == 0:
                print("Successfully fill in all the NaN values of ",  i , "-th patient!")
            else:
                print("Fail to fill in all the NaN values of ", i, "-th patient!")
                    #X[i] = X[i].fillna(X_train[i].mean())

Successfully fill in all the NaN values of  46 -th patient!
Successfully fill in all the NaN values of  121 -th patient!
Successfully fill in all the NaN values of  284 -th patient!
Successfully fill in all the NaN values of  298 -th patient!
Successfully fill in all the NaN values of  304 -th patient!
Successfully fill in all the NaN values of  315 -th patient!
Successfully fill in all the NaN values of  341 -th patient!
Successfully fill in all the NaN values of  386 -th patient!
Successfully fill in all the NaN values of  432 -th patient!
Successfully fill in all the NaN values of  478 -th patient!
Successfully fill in all the NaN values of  505 -th patient!
Successfully fill in all the NaN values of  587 -th patient!
Successfully fill in all the NaN values of  647 -th patient!
Successfully fill in all the NaN values of  649 -th patient!
Successfully fill in all the NaN values of  786 -th patient!
Successfully fill in all the NaN values of  908 -th patient!
Successfully fill in all 

In [132]:
X_testpd.to_csv ('test_set_all_filled.csv', index = False, header=True)

In [133]:
X_testpd.head()

,HR,O2Sat,Temp,MAP,Resp,Age,Gender,ICULOS,SepsisLabel,Patient_id
0,88.0,100.0,36.3,66.0,16.0,58.0,1,3,0,0
1,72.0,100.0,36.5,79.5,16.5,58.0,1,4,0,0
2,78.0,100.0,36.7,68.0,16.0,58.0,1,5,0,0
3,61.0,100.0,36.8,65.0,16.0,58.0,1,6,0,0
4,74.0,100.0,37.2,72.0,17.0,58.0,1,7,0,0


In [134]:
X_testpd.loc[test['Patient_id'] == 46] 
#Xi.columns[Xi.isna().any()].tolist()

,HR,O2Sat,Temp,MAP,Resp,Age,Gender,ICULOS,SepsisLabel,Patient_id
1767,62.5,100.0,35.17,56.0,18.605435,40.13,1,1,0,46
1768,62.5,100.0,35.17,56.0,18.605435,40.13,1,2,0,46
1769,65.0,100.0,35.17,53.0,18.605435,40.13,1,3,0,46
1770,72.0,99.5,35.17,50.5,18.605435,40.13,1,4,0,46
1771,73.0,100.0,35.39,54.0,18.605435,40.13,1,5,0,46
...,...,...,...,...,...,...,...,...,...,...
1909,103.0,93.0,37.78,63.0,18.605435,40.13,1,143,1,46
1910,100.0,93.0,37.78,62.0,18.605435,40.13,1,144,1,46
1911,100.0,95.0,37.78,58.0,18.605435,40.13,1,145,1,46
1912,104.0,95.0,37.78,65.0,18.605435,40.13,1,146,1,46


In [136]:
X_testpd.isnull().values.any()

False